## 最初に次の二つのセルを実行しておいてください

In [ ]:
import sys
sys.path.append('/home/jovyan/qc-workbook-lecturenotes/ja')
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Math
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, IBMQ, Aer, transpile
from qiskit.circuit import Parameter
from qiskit.tools.monitor import job_monitor
from qiskit.providers import JobStatus
from qiskit.providers.ibmq import least_busy, IBMQProviderError
from qc_workbook.show_state import statevector_expr
from qc_workbook.optimized_additions import optimized_additions
from qc_workbook.utils import operational_backend, find_best_chain
from qc_workbook.dynamics import make_heisenberg_circuits, plot_heisenberg_spins

In [ ]:
n_spins = 5
M = 10
omegadt = 0.1

IBMQ.load_account()

try:
    provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='qc-training22')
except IBMQProviderError:
    provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    
backend_list = provider.backends(filters=operational_backend(min_qubits=n_spins, min_qv=32))
backend = least_busy(backend_list)

print(f'Job will run on {backend.name()}')

circuits = make_heisenberg_circuits(n_spins, M, omegadt)

circuits_ibmq = transpile(circuits, backend=backend)

job_heisenberg = backend.run(circuits_ibmq, shots=8192)
print(f'Submitted job {job_heisenberg.job_id()}')

$\newcommand{\bra}[1]{\langle #1 |}$
$\newcommand{\ket}[1]{| #1 \rangle}$
$\newcommand{\upket}{\ket{\!\uparrow}}$
$\newcommand{\downket}{\ket{\!\downarrow}}$
$\newcommand{\rightket}{\ket{\!\rightarrow}}$
$\newcommand{\leftket}{\ket{\!\leftarrow}}$

# 第2回の講義の残り
## 量子テレポーテーション

In [ ]:
# まずは入力ビットを適当な状態にする回路を作る
# circuit.u (U3 gate)は3パラメータで一つの量子ビットを完全にコントロールするゲート
prep_circuit = QuantumCircuit(1, name='prep')
prep_circuit.u(0.7, 1.8, 2.1, 0)

reg_in = QuantumRegister(1, name='in')
reg_out = QuantumRegister(2, name='out')
res_in = ClassicalRegister(1)
res_ent = ClassicalRegister(1)

circuit = QuantumCircuit(reg_in, reg_out, res_in, res_ent)

# まずreg_inをprep_circuitの状態にする
circuit.append(prep_circuit, qargs=reg_in)

# reg_outはベル状態に用意する
circuit.h(reg_out[0])
circuit.cx(reg_out[0], reg_out[1])

# reg_inとreg_outの第一ビットをエンタングルさせる
circuit.cx(reg_in[0], reg_out[0])

# reg_inにアダマールゲートをかけ、測定する
circuit.h(reg_in[0])
circuit.measure(reg_in[0], res_in[0])

# reg_outのエンタングルしたビットも測定する
circuit.measure(reg_out[0], res_ent[0])

# reg_out[1]にreg_in, reg_entの測定結果に応じたゲートをかける
circuit.x(reg_out[1]).c_if(res_ent[0], 1)
circuit.z(reg_out[1]).c_if(res_in[0], 1)

circuit.draw('mpl')

入力ビットの状態は

In [ ]:
Math(statevector_expr(prep_circuit, state_label=r'\text{in}'))

回路の終状態は

In [ ]:
Math(statevector_expr(circuit, register_sizes=(1, 1, 1)))

## トランスパイル

[ibmq_limaのトポロジー](https://quantum-computing.ibm.com/services?services=systems&system=ibmq_lima)

In [ ]:
backend = provider.get_backend('ibmq_lima')

In [ ]:
# 論理ビット0と1の間でCXをやるだけの回路
circuit = QuantumCircuit(2)
circuit.cx(0, 1)

# 論理ビットをあえて離れた物理ビットにマップする
circuit = transpile(circuit, backend=backend, initial_layout=[0, 4])
circuit.draw('mpl')

In [ ]:
# Toffoliゲートの回路
circuit = QuantumCircuit(3)
circuit.ccx(0, 1, 2)

circuit.draw('mpl')

In [ ]:
# 実機に送られる回路
circuit = transpile(circuit, backend=backend)
circuit.draw('mpl')

# 計算をする量子回路の実装

## 量子フーリエ変換

$$
U_{\mathrm{QFT}} \ket{j} = \frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{2\pi i jk/2^n} \ket{k}
$$

QFTは量子回路で実装でき、線形なので、状態$\ket{\psi} = \sum_{j=0}^{2^n-1} c_j \ket{j}$に対しては

$$
\begin{split}
U_{\mathrm{QFT}} \ket{\psi} & = \frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n-1} c_j \sum_{k=0}^{2^n-1} e^{2\pi i jk/2^n} \ket{k} \\
& = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} \tilde{c}_k \ket{k} \quad \left( \tilde{c}_k = \sum_{j=0}^{2^n-1} c_j e^{2\pi i jk/2^n} \right)
\end{split}
$$

となり、振幅$\{c_j\}_j$の離散フーリエ変換が引き起こされることがわかります。

**例：$n=6$の時のQFT回路**

In [ ]:
num_qubits = 6

circuit = QuantumCircuit(num_qubits)

# 具体的にするため、入力状態を|23>とする
j = 23

## jの２進数表現で値が1になっているビットに対してXを作用させる -> 状態|j>を作る

for i in range(num_qubits):
    # j >> i: 整数jをi桁だけ右にビットシフトする。
    # 例えば j=13=001101 を i=2 右にシフトすると、000011 (右端の01が切り捨てられる)となる。
    # a & 1: 整数aと1とのビットごとのANDを取る。要するに一番右端のビットに1があるかどうかを見る。
    if ((j >> i) & 1) == 1:
        circuit.x(i)

circuit.barrier()

## ここからがQFT

# n-1から0まで標的ビットについてループ
for itarg in range(num_qubits - 1, -1, -1):
    # 標的ビットにアダマールゲートをかける
    circuit.h(itarg)
    # target - 1から0まで制御ビットについてループ
    for ictrl in range(itarg - 1, -1, -1):
        # 標的と制御ビットのインデックスに応じた角度で制御Pゲートをかける
        power = ictrl - itarg - 1 + num_qubits
        circuit.cp((2 ** power) * 2. * np.pi / (2 ** num_qubits), ictrl, itarg)
        
    # 回路図を見やすくするためにバリアを入れる
    circuit.barrier()

# 最後にビットの順番を反転させる
for i in range(num_qubits // 2):
    circuit.swap(i, num_qubits - 1 - i)
    
## ここまでがQFT

circuit.draw('mpl')

**状態ベクトルの表示**

In [ ]:
sqrt_2_to_n = 2 ** (num_qubits // 2)
amp_norm = (1. / sqrt_2_to_n, r'\frac{1}{%d}' % sqrt_2_to_n)
phase_norm = (2 * np.pi / (2 ** num_qubits), r'\frac{2 \pi i}{%d}' % (2 ** num_qubits))
expr = statevector_expr(circuit, amp_norm=amp_norm, phase_norm=phase_norm)
Math(expr)

## 量子フーリエ変換による足し算

$$
\ket{0}_{\mathrm{out}}\ket{b}_{\mathrm{in2}}\ket{a}_{\mathrm{in1}} \rightarrow \ket{a+b}_{\mathrm{out}}\ket{b}_{\mathrm{in2}}\ket{a}_{\mathrm{in1}}
$$

**足し算回路を組む関数**

In [ ]:
def setup_addition(circuit, reg1, reg2, reg3):
    # reg3にequal superpositionを生成
    # QuantumCircuitの1量子ビットゲートに対応するメソッド（circuit.hなど）に単一の量子ビットの代わりに
    # レジスタや量子ビットのリストを渡すと、含まれる全ての量子ビットに同じゲートをかけてくれる
    circuit.h(reg3)

    # 位相の単位（dphiの整数倍の位相をCPゲートでかけていく）
    dphi = 2. * np.pi / (2 ** reg3.size)

    # reg1とreg2それぞれの量子ビットで制御する
    for reg_ctrl in [reg1, reg2]:
        # 制御ビットに関するループ
        for ictrl, qctrl in enumerate(reg_ctrl):
            # reg3の標的ビットに関するループ
            for itarg, qtarg in enumerate(reg3):
                # C[P(phi)], phi = 2pi * 2^{ictrl} * 2^{itarg} / 2^{n3}
                circuit.cp(dphi * (2 ** (ictrl + itarg)), qctrl, qtarg)

    # 回路図を見やすくするためのバリア
    circuit.barrier()

    # Inverse QFT
    for j in range(reg3.size // 2):
        circuit.swap(reg3[j], reg3[-1 - j])

    for itarg in range(reg3.size):
        for ictrl in range(itarg):
            power = ictrl - itarg - 1 + reg3.size
            circuit.cp(-dphi * (2 ** power), reg3[ictrl], reg3[itarg])
        
        circuit.h(reg3[itarg])
        
print('Defined function setup_addition')

**9+13を計算する**

In [ ]:
a = 9
b = 13

# 入力の値を二進数表現できる最小のビット数を計算
n1 = np.ceil(np.log2(a + 1)).astype(int)
n2 = np.ceil(np.log2(b + 1)).astype(int)
n3 = np.ceil(np.log2(a + b + 1)).astype(int)

print(f'n1={n1}, n2={n2}, n3={n3}')

reg1 = QuantumRegister(n1, 'r1')
reg2 = QuantumRegister(n2, 'r2')
reg3 = QuantumRegister(n3, 'r3')

# QuantumCircuitは量子ビット数の代わりにレジスタを渡しても作成できる
circuit = QuantumCircuit(reg1, reg2, reg3)

# reg1を|a>にする
a_bits = np.unpackbits(np.asarray(a, dtype=np.uint8), bitorder='little')
for idx in np.nonzero(a_bits)[0]:
    circuit.x(reg1[idx])

# reg2を|b>にする
b_bits = np.unpackbits(np.asarray(b, dtype=np.uint8), bitorder='little')
for idx in np.nonzero(b_bits)[0]:
    circuit.x(reg2[idx])

# 足し算ルーチンを呼ぶ
setup_addition(circuit, reg1, reg2, reg3)

# 回路図を確認
circuit.draw('mpl')

**終状態**

In [ ]:
expr = statevector_expr(circuit, register_sizes=(n1, n2, n3))
Math(expr)

## 足し算の並列化

$$
\frac{1}{\sqrt{2^{n_1 + n_2}}} \sum_{j=0}^{2^{n_1}-1} \sum_{k=0}^{2^{n_2}-1} \ket{0}\ket{k}\ket{j} \rightarrow \frac{1}{\sqrt{2^{n_1 + n_2}}} \sum_{j=0}^{2^{n_1}-1} \sum_{k=0}^{2^{n_2}-1} \ket{j+k}\ket{k}\ket{j}
$$

In [ ]:
n1 = 4
n2 = 4
n3 = np.ceil(np.log2((2 ** n1) + (2 ** n2) - 1)).astype(int)

reg1 = QuantumRegister(n1, 'r1')
reg2 = QuantumRegister(n2, 'r2')
reg3 = QuantumRegister(n3, 'r3')

circuit = QuantumCircuit(reg1, reg2, reg3)

# reg1とreg2をequal superpositionにする
circuit.h(reg1)
circuit.h(reg2)

setup_addition(circuit, reg1, reg2, reg3)
    
expr = statevector_expr(circuit, register_sizes=(n1, n2, n3), amp_norm=(1. / np.sqrt(2 ** (n1 + n2)), r'\frac{1}{\sqrt{2^{n_1 + n_2}}}'))
Math(expr)

## シミュレータでの実行

In [ ]:
qasm_simulator = Aer.get_backend('qasm_simulator')

# 回路に測定を加えてトランスパイルする
circuit.measure_all()
circuit = transpile(circuit, backend=qasm_simulator)

job = qasm_simulator.run(circuit, shots=20)
counts = job.result().get_counts()

heights = []
labels = []

for key, value in counts.items():
    heights.append(value)

    # countsのキーはひとつなぎの二進数なので、出力, 入力2, 入力1の値が読み取れるように切り分ける
    # 4 + 4 桁なら
    #  00110 0101 0001 -> 6 = 5 + 1
    #  n3    n2   n1
    x1 = int(key[-n1:], 2) # last n1 digits
    x2 = int(key[-n1 - n2:-n1], 2) # next-to-last n2 digits
    x3 = int(key[:-n1 - n2], 2) # first n3 digits
    labels.append('{} + {} = {}'.format(x1, x2, x3))

x = np.linspace(0., len(labels), len(labels), endpoint=False)

# 棒グラフをプロット
plt.bar(x, heights, width=0.5)

# ビジュアルを調整
plt.xticks(ticks=(x - 0.2), labels=labels, rotation=70)
plt.tick_params('x', length=0.)

# 量子ダイナミクスシミュレーション

## $\exp \left(\frac{i\theta}{2} \sigma^{X,Y,Z}_j \sigma^{X,Y,Z}_{j+1}\right)$の実装

In [ ]:
# Rzzの回路
circuit = QuantumCircuit(QuantumRegister(2, 'spin'))
circuit.cx(0, 1)
circuit.rz(Parameter(r'-$\theta$'), 1)
circuit.cx(0, 1)
circuit.draw('mpl')

In [ ]:
# equal superpositionにRzzをかけてみる
theta = 0.2

circuit = QuantumCircuit(QuantumRegister(2, 'spin'))

circuit.h([0, 1])

circuit.cx(0, 1)
circuit.rz(-theta, 1)
circuit.cx(0, 1)

Math(statevector_expr(circuit, phase_norm=None, amp_norm=(0.5, r'\frac{1}{2}'), binary=True))

In [ ]:
# Rxxの回路
circuit = QuantumCircuit(QuantumRegister(2, 'q'))
# 基底の変換（Xの固有ベクトルをZの固有ベクトルに）
circuit.h(0)
circuit.h(1)
# Rzz
circuit.cx(0, 1)
circuit.rz(Parameter(r'-$\theta$'), 1)
circuit.cx(0, 1)
# 基底の逆変換（Zの固有ベクトルをXの固有ベクトルに）
circuit.h(0)
circuit.h(1)

circuit.draw('mpl')

In [ ]:
# Ryyの回路
circuit = QuantumCircuit(QuantumRegister(2, 'q'))
# 基底の変換（Yの固有ベクトルをXの固有ベクトルに）
circuit.p(-np.pi / 2., 0)
circuit.p(-np.pi / 2., 1)
# 基底の変換（Xの固有ベクトルをZの固有ベクトルに）
circuit.h(0)
circuit.h(1)
# Rzz
circuit.cx(0, 1)
circuit.rz(Parameter(r'-$\theta$'), 1)
circuit.cx(0, 1)
# 基底の逆変換（Zの固有ベクトルをXの固有ベクトルに）
circuit.h(0)
circuit.h(1)
# 基底の逆変換（Xの固有ベクトルをYの固有ベクトルに）
circuit.p(np.pi / 2., 0)
circuit.p(np.pi / 2., 1)

circuit.draw('mpl')

## ハイゼンベルグモデルの回路

初期状態$\rightket\upket\upket\upket\upket$（量子レジスタでの表現では右から書くので$\ket{0}\ket{0}\ket{0}\ket{0}\frac{1}{\sqrt{2}}(\ket{0} + \ket{1})$）

In [ ]:
n_spins = 5
M = 10
omegadt = 0.1

circuits = []

circuit = QuantumCircuit(n_spins)

# 第0ビットを 1/√2 (|0> + |1>) にする
circuit.h(0)

# Δtでの時間発展をM回繰り返すループ
for istep in range(M):
    # ハミルトニアンのn-1個の項への分解に関するループ
    for jspin in range(n_spins - 1):
        # ZZ
        circuit.cx(jspin, jspin + 1)
        circuit.rz(-omegadt, jspin + 1)
        circuit.cx(jspin, jspin + 1)

        # XX
        circuit.h(jspin)
        circuit.h(jspin + 1)
        circuit.cx(jspin, jspin + 1)
        circuit.rz(-omegadt, jspin + 1)
        circuit.cx(jspin, jspin + 1)
        circuit.h(jspin)
        circuit.h(jspin + 1)

        # YY
        circuit.p(-np.pi / 2., jspin)
        circuit.p(-np.pi / 2., jspin + 1)
        circuit.h(jspin)
        circuit.h(jspin + 1)
        circuit.cx(jspin, jspin + 1)
        circuit.rz(-omegadt, jspin + 1)
        circuit.cx(jspin, jspin + 1)
        circuit.h(jspin)
        circuit.h(jspin + 1)
        circuit.p(np.pi / 2., jspin)
        circuit.p(np.pi / 2., jspin + 1)

    # この時点での回路のコピーをリストに保存
    # measure_all(inplace=False) はここまでの回路のコピーに測定を足したものを返す
    circuits.append(circuit.measure_all(inplace=False))
    
print(f'{len(circuits)} circuits created')

qasm_simulatorで実行

In [ ]:
# Define the initial statevector
initial_state = np.zeros(2 ** n_spins, dtype=np.complex128)
initial_state[0:2] = np.sqrt(0.5)

shots = 100000

qasm_simulator = Aer.get_backend('qasm_simulator')

circuits = transpile(circuits, backend=qasm_simulator)
sim_job = qasm_simulator.run(circuits, shots=shots)
sim_counts_list = sim_job.result().get_counts()
   
plot_heisenberg_spins(sim_counts_list, n_spins, initial_state, omegadt, add_theory_curve=True)

講義の最初に投げたジョブは同じ内容を実機で実行するもの

In [ ]:
## ノートブックを再起動したなどの理由で、もともとのジョブオブジェクトにアクセスできない場合
## __backend_name__ と __job_id__ はIBM Quantumのページの Recent jobsから確認できる
# backend = provider.get_backend('__backend_name__')
# job_heisenberg = backend.retrieve_job('__job_id__')

if job_heisenberg.status() == JobStatus.DONE:
    counts_list = job_heisenberg.result().get_counts()
    plot_heisenberg_spins(counts_list, n_spins, initial_state, omegadt, add_theory_curve=True)
else:
    print('Sorry, come back later!')